<a href="https://colab.research.google.com/github/kytk/AI-MAILs/blob/main/python_2_pandas_seaborn.ipynb?hl=ja" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 医療従事者のためのPython: PandasとSeaborn

根本清貴 (筑波大学医学医療系精神医学)

Ver.20240703

## 参考にした資料
- [Pandas公式ドキュメント(英語)](https://pandas.pydata.org/docs/index.html)
- [Seaborn公式ドキュメント(英語)](https://seaborn.pydata.org/tutorial/introduction)

## 本日使用するデータ
- 糖尿病データセット https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html
- 配布されているファイルの列名だけ編集し、2つに分割してExcelファイルに変換
- diabetes_demographics.xlsx: 年齢、性別、BMI、平均血圧
- diabetes_data.xlsx: T-Cho, LDL, HDL, T-Cho/HDL, TGの対数, 血糖, Y(1年間での進行度)

## 本セクションの目標
- Pandasを使ってExcelファイルを読み込むことができるようになる
- Pandasを使って任意の列や行を抽出できるようになる
- Pandasを使って2つのファイルを結合することができるようになる
- Seabronを使って様々なグラフが描けるようになる

## 本日の内容
1. PandasとSeabornの概要
2. Pandasの基礎
   - データの読み込み
   - データの表示
   - データの操作
   - データの結合
   - 欠損値の扱い
3. Seabornの基礎
   - 基本的なグラフ
   - カスタマイズ
4. 練習問題


## 1. Pandas と Seaborn の概要
- Pandas
    - Pythonでデータ分析およびデータ操作をするためのツール 
    - "Pandas is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool, built on top of the Python programming language."
- Seaborn
    - Pythonで統計データからグラフを簡便に作成するためのツール 
    - "Seaborn is a library for making statistical graphics in Python."

### 1.0. Pandas と Seaborn のインストール (今回は不要)
- 自分でPython環境を整える際は、以下で Pandas と Seaborn をインストールする

    ```
    pip install pandas seaborn
    ```

### 1.1. Pandas と Seaborn, および os のインポート
- pandas は pd としてインポートされることが多い
- seaborn は sns としてインポートされることが多い
- ファイルを確認するために、os モジュールもインポートする


In [ ]:
# pandas は pd としてインポートされることが多い
import pandas as pd

# seaborn は sns (seaborn name space) としてインポートされることが多い
import seaborn as sns

# ファイルを確認するのは、os モジュールをインポートする
import os

### 1.2. Google colaboratory へのデータの読み込み
- Google colaboratoryで作業をする時、通常は、左にある「ファイル」の下にデータをドラッグ & ドロップする
- そうすると、'ファイル名' でアクセスできる
- 今は、データをダウンロードして使用する
- 下のセルを実行すると、3つのxlsxファイルがダウンロードされる
- **注意**: Google colaboratoryでの作業内容は一定時間で消える。データを保存するためには、適宜作業結果のダウンロードなどが必要

<img src="https://www.nemotos.net/nb/img/colabo_files.png" width=300>

In [ ]:
# 本日使うデータをダウンロードする
# ファイル名は diabetes_demographics.xlsx, diabetes_demographics_short.xlsx, diabetes_data.xlsx
# 
# (参考: 本講義においては本質的な事柄ではないのでわからなくても特に気にしなくて大丈夫です)
# ! はPythonからシェルで動かすプログラムを実行したい時につける
# [[ -f diabetes_demographics.xlsx ]] は、「diabetes_demographics.xlsx というファイルがあるか？」というシェルスクリプトのテスト文
# || は、戻り値が False ならば…　というコマンド
# wget はダウンロードするLinuxのプログラム

![[ -f diabetes_demographics.xlsx ]] || wget https://raw.githubusercontent.com/kytk/AI-MAILs/main/data/diabetes_demographics.xlsx
![[ -f diabetes_data.xlsx ]] || wget https://raw.githubusercontent.com/kytk/AI-MAILs/main/data/diabetes_data.xlsx
![[ -f diabetes_data_short.xlsx ]] || wget https://raw.githubusercontent.com/kytk/AI-MAILs/main/data/diabetes_data_short.xlsx


In [ ]:
# osモジュールの中にある listdir() 関数で今のディレクトリの下にあるファイル一覧を表示できる
# 以下の3つのファイルがあることを確認
#   diabetes_demographics.xlsx
#   diabetes_data.xlsx
#   diabetes_data_short.xlsx

os.listdir()

## 2. Pandasの基礎

### 2.1. Pandasでできることの例
- 表を扱う
- 表の記述統計量を出力する
- 表の一部を抽出する
- 複数の表を結合する
- 複数の列の情報から新しい列を生成する
- PandasのデータフレームはSeabornでのグラフ作成にそのまま使用できる

### 2.2. Pandasの必要性: データの前処理とクリーニング
- データの前処理は、データ解析において重要なステップ
- Pandasを使用すると、欠損値の処理、データ型の変換、重複データの削除などを行うことができる

### 2.3. Pandas の用語: 「データフレーム Data Frame」
- 一般的な表をイメージすればよい
- Pandasでは、ひとつの表を「データフレーム」と呼ぶ
- 行: row; 列: column
- dataframe という名前を略して、 'df' という変数に代入することが多い

<img src="https://www.nemotos.net/nb/img/pandas_01.png" width=400>
図: pandas の公式ドキュメントから引用

### 2.4. Pandas へのデータの読み込み
- Pandas は、excelファイルやcsvファイルを読み込むことができる
- Excel は、`pd.read_excel('excelファイル')` で読み込むことができる
- CSVは、`pd.read_csv('csvファイル')` で読み込むことができる
- IDなどは、インデックスとして指定すると取り回しが楽になる

- diabetes_demographics.xlsx

    <img src="https://www.nemotos.net/nb/img/diabetes_demographics_screenshot.png" width=300>

- diabetes_data.xlsx

    <img src="https://www.nemotos.net/nb/img/diabetes_data_screenshot.png" width=450>


In [ ]:
# diabetes_demographics.xlsx を df_demographics として読み込む。第0列をインデックス列とする
df_demographics = pd.read_excel('diabetes_demographics.xlsx',index_col=0)

# diabetes_data.xlsx を df_data として読み込む。第0列をインデックス列とする
df_data = pd.read_excel('diabetes_data.xlsx',index_col=0)

# diabetes_data_short.xlsx を df_data_short として読み込む。第0列をインデックス列とする
df_data_short = pd.read_excel('diabetes_data_short.xlsx',index_col=0)

#### データフレームの概要を確認
- データフレームにどのような情報が入っているかは pandasデータフレームのメソッド、info() で知ることができる

In [ ]:
# df_demographics についての概要を知る
df_demographics.info()

#### カテゴリカルデータの扱い
- 今、SEXは int64 と整数型となっている。
- これを以下のようにカテゴリカルデータと変更する方法もある
    ```
    # データ型の変換
    df_demographics['SEX'] = df_demographics['SEX'].astype('category')
    ```
- しかし、この場合、その後エラーが出やすいので、ここでは意図的に変更しない
- この後出てくる記述統計量は注意が必要


#### Pandasデータフレームの型
- pandasのデータフレームは独自の型で定義されている
- pandasデータフレームの型は `type()` 関数でわかる

In [ ]:
# データフレームの型
# pandas.core.frame.DataFrame型
type(df_demographics)

### 2.5. データの表示
- 読み込んだデータフレームを df とすると、head() メソッドを使うことで、最初の5行を表示できる
    - head(10)とすると、10行が表示される
- 表の大きさは、df.shape で確認できる

In [ ]:
# df_demographics の最初の5行を確認

# 今、ID列がインデックス列になっているので、IDだけ段がひとつずれていることに注意
df_demographics.head()

In [ ]:
# インデックス列として指定しないとどうなるかも確認
# IDがひとつの変数として読み込まれる
# インデックスは一番左の 0, 1, 2 ...

df_demo_without_index = pd.read_excel('diabetes_demographics.xlsx')
df_demo_without_index.head()

In [ ]:
# df_demographics の表の大きさを確認
# インデックス(ID)以外は442行4列
df_demographics.shape

In [ ]:
# df_data の最初の5行を確認
df_data.head()

In [ ]:
# df_data の表の大きさを確認
# インデックス(ID)以外は442行7列
df_data.shape

In [ ]:
# df_data_short の最初の5行を確認
# IDが連番になっていないデータ
df_data_short.head()

In [ ]:
# df_data_short の表の大きさを確認
# インデックス(ID)以外は252行7列
df_data_short.shape

### 2.6. データの記述統計量
- Pandasはデータ解析ツールとうたっているだけあり、記述統計量は簡単に求められる


In [ ]:
# df.describe() で各項目の記述統計量が求められる
# 連続値の n数、平均、標準偏差、最小値、25パーセンタイル、50パーセンタイル、75パーセンタイル、最大値が一覧で出力される
df_demographics.describe()

- groupby() メソッドを使うと、群毎の記述統計量を計算できる

In [ ]:
# groupby() メソッドで、性別毎の平均を計算する
df_demographics.groupby(by='SEX').mean()

- corr() メソッドを使うと、相関係数を計算できる

In [ ]:
# データの相関も corr() メソッドで簡単に求められる
# df_data の各列の相関を求める

# df_data の3行を表示する
df_data.head(3)

In [ ]:
# 相関を求める
df_data.corr()

### 2.7. データの操作 (1)
#### 列の抽出
- Pandas では、カラム名を使って、データから特定の列を抽出できる
- Pythonは要素を取り出したい時は、[] を使う。この原則に従って、df['列名'] とすることで、特定の列を取り出せる

In [ ]:
# df_data の列名は、df_data.columns　で求められる
df_data.columns

In [ ]:
# 'T-Cho' の列だけを抽出したかったら、df_demographics['T-Cho']とする
df_data['T-Cho']

In [ ]:
# 取り出した列だけの平均を求めることもできる
df_data['T-Cho'].mean()

- 複数のカラムを取り出したい時は、カラム名をリストで作成し、それをdf[[リスト]]とする
- リストはもともと [] でくくっており、それを、要素を抽出する [] でくくるので、結果的に [[]] となっていることに注意

In [ ]:
# T-Cho, LDL, HDL, Glu, Y を取り出したい
# 考え方として、まず、リストを作成する
# そのリストを df_data[] の中に入れる
# ['T-Cho', 'LDL', 'HDL', 'Glu', 'Y']
df_data[['T-Cho', 'LDL', 'HDL', 'Glu', 'Y']]

In [ ]:
# ここでも同じように平均値を求められる
# 連続値のみ平均値が計算される
df_data[['T-Cho', 'LDL', 'HDL', 'Glu', 'Y']].mean()

#### 演習 1
- pandasで複数のカラムを抽出したい時、リストを使うことの意義を感じてもらうために以下を実行してください
- 'T-Cho', 'LDL', 'Glu' の3項目を a というリストに代入してください
- df_data[a] を実行してみてください


In [ ]:
# あなたの回答
# 'T-Cho', 'LDL', 'Glu' を a というリストに代入
a =
# a を表示
# ['T-Cho', 'LDL', 'Glu'] と表示されれば正解
print(a)

In [ ]:
# df_data[a] を実行
df_data[a]

In [ ]:
# 回答例
a = ['T-Cho', 'LDL', 'Glu']
print(a)
df_data[a]
# a そのものが [] でくくられているので、df[a] は df[['Age', 'Subject_Type', 'CSF']] と同義
# 慣れるまでは、まず、リストを作成して、それをdf[]に入れるように意識するのがよいかもしれない

#### 行の抽出
- 行を抽出したい場合には、loc (**loc**ation) やiloc (**i**nteger **loc**ation)を使う
- loc の後には、取り出したいインデックスの値を記載する
- iloc の後には、取り出したい行番号を記載する。スライシングで複数行を指定できる

In [ ]:
# loc は 特定のインデックスの行を取り出せる
# 今、インデックスはID
# ID sub400 を取り出す
df_data.loc['sub400']


In [ ]:
# IDをリストで指定すると、複数行取り出せる
df_data.loc[['sub400', 'sub410']]

In [ ]:
# iloc は 行番号で指定した行を取り出せる
# スライシングはPythonの基本と同じ
# 最初の行の行番号は0
# 行が1行目からはじまるとして、3行目から5行目を取り出したい時、
# 行番号が0から始まるとすると、行番号の2から4を取り出すということなので、
# 2行以上5行未満と考える
df_data.iloc[2:5]

In [ ]:
# df_data.head() で最初の5行を表示することで確認
df_data.head()

### 2.8. データの結合
#### データフレームの横方向への結合
- pandasの得意なことに、表を結合することがある
- 2つの表でIDが完全に一致しない時に、Excelではかなり手間がかかる
- pd.merge() 関数を使うことで、表（データフレーム）を横方向に結合できる
- 2つのデータフレームに共通するキーを `on='キー'` を使って指定する

### 内部結合と外部結合
- merge には内部結合と外部結合がある
    - *正確には外部結合には、左側、右側、完全の3つがあるが今回は完全外部結合のみ説明
- 内部結合は2つのデータに共通するもののみ結合する(下図ではID04, ID05, ID06のみが結合される)
- 外部結合は2つのデータのすべてが結合する
- 数学で言えば、2つのグループA, B に対し
    - 内部結合は積集合: A∩B
    - 外部結合は和集合: A∪B

<img src="https://www.nemotos.net/nb/img/pandas_02.png" width=300>


- デフォルトは、共通するキーがある行だけが結合される(内部結合)

- 今、df_demographics は 442行あるのに対し、df_data は 442行、df_data_short は254行ある (研究データで一部のデータがとれていないイメージ)

- 3つのことを行う
    - df_demographics (442行) と df_data (442行) を結合する（Excelでも簡単にできる）
    - df_demographics (442行) と df_data_short (254行) を内部結合する（Excelだと大変）
    - df_demographics (442行) と df_data_short (254行) を外部結合する (Excelだと大変)



In [ ]:
# df_demographics と df_data を 'ID' キーを使って連結し、df というデータフレームを生成する
df = pd.merge(df_demographics, df_data, on='ID')

In [ ]:
# df の最初の5行だけ表示する
# demographics の列と data の列が結合したことに着目
df.head()

In [ ]:
# df の大きさを確認する
# 442行11列
df.shape

In [ ]:
# df_demographics と df_data_short を 'ID' キーを使って連結し、df_short というデータフレームを生成する
df_short = pd.merge(df_demographics, df_data_short, on='ID')

In [ ]:
# df_short の大きさを確認
# 254行11列
df_short.shape

In [ ]:
# df_short の最初の5行だけ表示する
# IDが少なくなっていることに着目
df_short.head()

In [ ]:
# df_demographics と df_data_short に対して 外部結合を行い、df_short_outer というデータフレームを生成する
# 外部結合には、how='outer' と記載する
df_short_outer = pd.merge(df_demographics, df_data_short, how='outer', on='ID')

In [ ]:
# df_short_outer の大きさを確認
# 442行11列
df_short_outer.shape

In [ ]:
# df_outer の最初の5行を確認する
# 血液データがないものについては、血液データの項目が NaN となっていることに着目
df_short_outer.head()

### 2.9. 欠損値の扱い
- df_short_outer を使って欠損値の扱いを学ぶ
- データフレームの isnull() メソッドは値がないところを True に それ以外は False とする

In [ ]:
df_short_outer.isnull()

In [ ]:
# df_short_outer.isnull() にさらに メソッド sum() を使うことで、値がない個数を知ることができる

# 今、血液データは 各項目190個欠損値があることがわかる
df_short_outer.isnull().sum()

In [ ]:
# 欠損値に対しては、「ドロップする」か「補完する」の2つ

# まずはドロップする場合
# dropna() メソッドを引数なしで実行する
df_dropped = df_short_outer.dropna()

In [ ]:
# もともと442あったものが、NaN (Not a Number) が 190あったので 442 - 190 = 252 となる
df_dropped.shape

In [ ]:
# 補完する場合は fillna() メソッドの引数に 平均値などを指定する
df_filled = df_short_outer.fillna(df_short_outer.mean())

In [ ]:
# NaN がなくなる
df_filled.head()

In [ ]:
# 元のものと比較すると、NaN が各列の平均値で均一に埋められているのがわかる
# もちろん、現実には、こんなにも欠損値があったらこの方法は不適切だが、ここは例として紹介
df_short_outer.head()

### 2.10. データの操作 (2)
#### 条件抽出
- 条件に合致するものだけを抽出することもできる
- df['AGE']>40 は、年齢が40歳よりも大きい人をTrueかFalseで返す
- これを df[] の中にいれることでこの条件に合致する人の一覧を作成できる

In [ ]:
# df['AGE'] > 40 は True か False を返す
df['AGE'] > 40

In [ ]:
# 上記を dfのインデックス の中に入れてあげることで、Trueのものだけ抽出できる
df[df['AGE'] > 40]

In [ ]:
# &, |, ~ を使うことで、複数条件を作れる
# ルール
#   &, |, ~ のどれかを使わなければいけない。and, or, notは使えない
#  ひとつひとつの条件をそれぞれ () でくくらないといけない

# 50歳より大きく女性
(df['AGE'] > 50) & (df['SEX'] == 2)

In [ ]:
# この条件をdf[] の中に入れてみる
df[(df['AGE'] > 50) & (df['SEX'] == 2)]

#### 演習 2
- 'AGE' が 40歳以上、かつ、'BMI' が30以上のデータを抽出したいと思います。理解を深めるために以下を実行してください
    - 'AGE' が 40歳以上 という条件式を作って、condition1 に代入してください 
    - 'BMI' が 30以上 という条件式を作って、 condition2 に代入してください
    - condition1 かつ condition2 という条件式を作って、 b に代入してください
    - df[b] を実行し、df_b というデータフレームを作成してみてください
    - df_b の最初の10行を表示してください
    - df_b の記述統計量を計算してください

In [ ]:
# 'AGE' が40歳以上 かつ 'BMI' が 30以上
# ヒント: 40以上は　 >= 40
# ヒント: かつ　は &

# condition1: 'AGE' が40歳以上
condition1 =

# condition1を表示
condition1.head()

In [ ]:
# condition2: 'BMI' が30以上
condition2 =
# condition2を表示
condition2.head()

In [ ]:
# b: condition1 かつ condition2
b =

# b を表示
b

In [ ]:
# df[b] を実行し、df_b というデータフレームを作成
df_b = df[b]

# df_b の最初の10行を表示
# 以下に記載してください (ヒント head)


In [ ]:
# df_bの記述統計量を計算
# 以下に記載してください (ヒント describe)


In [ ]:
# 回答例
condition1 = df['AGE'] >= 40
print('condition1')
print(condition1.head())

condition2 = df['BMI'] >= 30
print('condition2')
print(condition2.head())

b = condition1 & condition2
print('b')
print(b.head())

df_b = df[b]
df_b.head(10)

In [ ]:
# 回答例
# 記述統計量を計算
df_b.describe()

#### 計算したものを新たな列に追加
- Pandas では、項目を計算して新たな列を生成することが簡単にできる
- 計算だけでなく、Bool値なども追加できる

In [ ]:
# GlucoseのZ値を計算
# (Glu - Gluの平均) / Gluの標準偏差
df['Glu_Z'] = (df['Glu'] - df['Glu'].mean())/df['Glu'].std()

In [ ]:
df.head()

In [ ]:
# T-choが220より大きいか否かをHCという列に作成する
df['HC'] = df['T-Cho']>220

In [ ]:
# 最後の列に HC ができていることに着目
df.head()

#### ファイルの書き出し
- pandas は csv や excel に簡単に保存できる
- メソッド 'to_ファイルの種類' を使う
- df.to_csv('ファイル名')
- df.to_excel('ファイル名')


In [ ]:
# 今の df を 'diabetes.xlsx' というファイル名で出力する
# Google colaboratory の中に生成される。それはダウンロードできる
df.to_excel('diabetes.xlsx')

## 3. Seabornの基礎
- Seaborn は様々なグラフを簡単なコマンドで描ける
- これにより、データの関係性を図示できる


In [ ]:
# seaborn のテーマを使うには、次のコマンドを実行する
sns.set_theme()

#### Seabronの一般的なルール
- snsの後にグラフごとに準備されているメソッドを記載する
    - sns.scatterplot()
- 引数として、以下を指定する
    - data=データフレーム
    - x='X軸に使いたい項目'
    - y='Y軸に使いたい項目'
    - hue='色分けしたい項目'
    - style='プロットの形を分けたい項目'

### 3.1. 基本的なグラフ

In [ ]:
# df のおさらい
df.head()

#### A. scatterplot (散布図)
- グラフの目的: 2つの連続変数間の関係性を視覚化する


In [ ]:
sns.scatterplot(data=df, x='AGE', y='BMI', hue='SEX')

In [ ]:
# 色を指定したい場合、変数と色の対応表を辞書型で作成し、それを palette で指定する
color_dict = {1: 'blue', 2: 'orange'}  # 1 は男性、2 は女性の例
sns.scatterplot(data=df, x='AGE', y='BMI', hue='SEX', palette=color_dict)

#### B. regplot (回帰直線付き散布図)
- グラフの目的: 2つの連続変数間の関係性とその回帰直線を視覚化する

In [ ]:
sns.regplot(data=df, x='AGE', y='BMI')

In [ ]:
# 信頼区間はデフォルトでは 95%
# 99%に変えたい場合は ci=99 と指定する
# 信頼区間が不要な場合は ci=None
sns.regplot(data=df, x='BMI', y='Glu', ci=99)

#### C. histplot (ヒストグラム)
- グラフの目的: 変数の分布を視覚化する。カーネル密度推定（KDE）も追加できる


In [ ]:
# 複数のヒストグラムを同時に描くことができる
sns.histplot(df[['T-Cho','LDL']])

In [ ]:
# カーネル密度推定を追加
sns.histplot(data=df, x='Glu', kde=True)

In [ ]:
# (参考)
# 正規分布曲線をプロットするのは、Seabornだけでは残念ながらできない
# 以下のようにMatplotlibを使用する

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm

# ヒストグラムをプロット
sns.histplot(data=df, x='Glu', stat='density', bins=20)

# 正規分布曲線を計算
mean = df['Glu'].mean()
std = df['Glu'].std()
x = np.linspace(df['Glu'].min(), df['Glu'].max(), 100)
p = norm.pdf(x, mean, std)

# 正規分布曲線をプロット
plt.plot(x, p, 'k', linewidth=1)
plt.title('Histogram and Normal Distribution Curve')
plt.show()


#### D. boxplot (箱ひげ図)
- グラフの目的: データの分散や異常値を確認する
   

In [ ]:
sns.boxplot(df[['T-Cho', 'LDL', 'HDL', 'Glu']])

#### E. violinplot (バイオリンプロット)
- グラフの目的: データの分布と密度を確認する

In [ ]:
sns.violinplot(df[['T-Cho', 'LDL', 'HDL', 'Glu']])

#### F. swarmplot (スウォームプロット/ビースウォームプロット)
- グラフの目的: データの分布を個別の点として視覚化し、データポイントの重なりを回避する際に使用する

In [ ]:
sns.swarmplot(data=df, x='SEX', y='BMI', hue='HC')

#### G. pairplot (ペアプロット)
- グラフの目的: 複数の変数間の関係を一度に可視化する

In [ ]:
sns.pairplot(df[['AGE','BMI','T-Cho','LDL','Glu']])

#### H. heatmap (ヒートマップ)
- グラフの目的: 相関行列を視覚化し、変数間の相関を理解する際に使用する

In [ ]:
# まず、変数同士の相関係数を計算し、correlationという変数に代入する
correlation = df[['AGE','BMI','T-Cho','LDL','Glu']].corr()
# correlationの内容を確認する
correlation

In [ ]:
# correlation をヒートマップで表示する
# annot=True で数値を表示できる
# fmt='.2f' は小数点第2位までという意味
sns.heatmap(correlation, annot=True, fmt='.2f')

#### I. countplot (カウントプロット)
- グラフの目的: カテゴリデータの分布を視覚化する

In [ ]:
sns.countplot(data=df, x='SEX')

#### J. clustermap (クラスターマップ)
- グラフの目的: データのクラスターを視覚化し、データのパターンや類似性を探る

In [ ]:
sns.clustermap(df.corr(), annot=True, cmap='coolwarm')

#### K. インタラクティブなプロット
- グラフの目的: インタラクティブにプロットを操作し、データの関係性を探索する


In [ ]:
from ipywidgets import interact

def plot_scatter(x, y):
    sns.scatterplot(data=df, x=x, y=y)

interact(plot_scatter, x=df.columns, y=df.columns)

#### L. relplot (リレーションプロット)
- グラフの目的: 複数の変数を同時に視覚化し、変数間の関係性を包括的に理解する際に使用する。色 (hue),形 (style), サイズ (size)でグループ分けが可能

    ```
    sns.relplot(data=データフレーム,
                x=x軸に使いたいカラム,
                y=y軸に使いたいカラム,
                hue=色を変えたいカラム,
                style=グラフのプロットの形状を変えたいカラム
                size=プロットの大きさを反映したいカラム)
    ```

In [ ]:
# シンプルに、BMI と 血糖 (Glu)の関係を図示する
sns.relplot(data=df, x='BMI',y='Glu')

In [ ]:
# hue を使うと、グループを色分けできる
# 今、高コレステロール血症の有無 でグループを色分けする
sns.relplot(data=df, x='BMI',y='Glu', hue='HC')

In [ ]:
# style を使うと、グループをプロットの形で判別できる
# 今、男性と女性の形状を変えたい
sns.relplot(data=df, x='BMI',y='Glu', hue='HC', style='SEX')

In [ ]:
# sizeを使うと、ひとつひとつのプロットの大きさを変えられる
# プロットに 年齢を反映させたい
sns.relplot(data=df, x="BMI", y="Glu",hue="HC",style="SEX",size="AGE")

## 4. 練習問題
1. dfデータフレームの'AGE'列の平均を計算してください
2. dfデータフレームの'BMI'列の最大値を求めてください
3. dfデータフレーム中で、'SEX'が1のデータのみ抽出してください
4. dfデータフレームから、AGE（年齢）とBMIの関係を示す散布図を作成してください。
5. 問4で作成した散布図に対して、SEX（性別）で色分けしてください。
6. dfデータフレームから、SEX（性別）ごとにBMIの分布を示す箱ひげ図を作成してください。
7. dfデータフレームから、SEX（性別）ごとにBMIの分布を示すバイオリンプロットを作成してください。
8. dfデータフレームのAGE, BMI, T-Cho, LDL, Gluの相関行列を計算してください。
9. 問8で作成した相関行列をヒートマップで視覚化し、各セルに相関係数を表示してください。


In [ ]:
# 問1
# ヒント: 2.6. 記述統計量


In [ ]:
# 問2
# ヒント: 2.6. 記述統計量

In [ ]:
# 問3
# ヒント: 2.7. データの操作


In [ ]:
# 問4
# ヒント: 3.1.A
  

In [ ]:
# 問5
# ヒント: 3.1.A


In [ ]:
# 問6
# ヒント: 3.1.D


In [ ]:
# 問7
# ヒント: 3.1.E


In [ ]:
# 問8
# ヒント: 2.6


In [ ]:
# 問9
# ヒント: 3.1.H


In [ ]:
#### 解答例
# 問1
df['AGE'].mean()

In [ ]:
# 問2
df[df['SEX'] == 1]

In [ ]:
# 問3
df['BMI'].max()

In [ ]:
# 問4
sns.scatterplot(data=df, x='AGE', y='BMI')

In [ ]:
# 問5
sns.scatterplot(data=df, x='AGE', y='BMI', hue='SEX')

In [ ]:
# 問6
sns.boxplot(data=df, x='SEX', y='BMI')

In [ ]:
# 問7
sns.violinplot(data=df, x='SEX', y='BMI')

In [ ]:
# 問8
correlation = df[['AGE', 'BMI', 'T-Cho', 'LDL', 'Glu']].corr()
correlation

In [ ]:
# 問9
sns.heatmap(correlation, annot=True, fmt='.2f', cmap='coolwarm')